In [29]:
import re
import random
import math
import binascii
import ast
import string
from time import time
import shutil
from PIL import Image
import numpy as np

# PRIMERS
KEY_PRIMER = "k_"
ROUNDS_NUM_PRIMER = "n_"
ROUNDS_CURRENT_PRIMER = "r_"
PARTITIONING_PRIMER = "r-"
GEN_CROSSOVER_PRIMER = "cd_"
CROSSOVER_TYPE_PRIMER = "t_"
SINGLE_POINT_CROSSOVER_PRIMER = "s_"
ROTATE_CROSSOVER_PRIMER = "ro_"
ROTATION_OFFSET_PRIMER = "o_"
ROTATION_TYPE_PRIMER = "t_"
MUTATION_PRIMER = "m_"
COMPLIMENT_MUTATION_PRIMER = "c_"
ALTER_MUTATION_PRIMER = "a_"
MUTATION_TABLE_PRIMER = "m-"
CHROMOSOME_PRIMER = "c-"

# Generate encoding tables domains
two_bit_list = ['00', '01', '10', '11']
dna_bases = ['A', 'C', 'G', 'T']

four_bit_list = ['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']
two_dna_bases = ['TA', 'TC', 'TG', 'TT', 'GA', 'GC', 'GG', 'GT', 'CA', 'CC', 'CG', 'CT', 'AA', 'AC', 'AG', 'AT']

# Encoding tables and their reversal
two_bits_to_dna_base_table = None
dna_base_to_two_bits_table = None

four_bits_to_two_dna_base_table = None
two_dna_base_to_four_bits_table = None

# Utility functions
def str2bin(s):
    """Convert string to binary."""
    return ''.join(bin(ord(c))[2:].zfill(8) for c in s)

def bin2str(bs):
    """Convert binary string to original string."""
    n = int(bs, 2)
    return binascii.unhexlify('%x' % n).decode('utf-8')

def byte2bin(byte_val):
    """Convert byte to binary string."""
    return bin(byte_val)[2:].zfill(8)

def bitxor(a, b):
    """Perform XOR on two binary strings."""
    return "".join([str(int(x) ^ int(y)) for (x, y) in zip(a, b)])

def divisors(n):
    """Find divisors of a number."""
    divs = [i for i in range(2, int(math.sqrt(n)) + 1) if n % i == 0]
    divs += [n // i for i in divs if i != n // i]
    return list(set(divs))

def generate_pre_processing_tables():
    """Generate encoding tables."""
    global two_bits_to_dna_base_table
    global dna_base_to_two_bits_table
    two_bits_to_dna_base_table = dict(zip(two_bit_list, dna_bases))
    dna_base_to_two_bits_table = {v: k for k, v in two_bits_to_dna_base_table.items()}

def generate_mutation_tables():
    """Generate mutation tables."""
    global four_bits_to_two_dna_base_table
    global two_dna_base_to_four_bits_table
    four_bits_to_two_dna_base_table = dict(zip(four_bit_list, two_dna_bases))
    two_dna_base_to_four_bits_table = {v: k for k, v in four_bits_to_two_dna_base_table.items()}

def group_bits(byte, step=2):
    """Group bits into specified step."""
    return [byte[i:i + step] for i in range(0, len(byte), step)]

def group_bases(dna_seq, step=2):
    """Group DNA bases into specified step."""
    return [dna_seq[i:i + step] for i in range(0, len(dna_seq), step)]

def generate_bits(byte_data):
    """Generate bits from byte data."""
    return [bit for byte in byte_data for bit in group_bits(byte)]

def binarized_data(data):
    """Convert string data to binary bits."""
    byte_data = [byte2bin(ord(c)) for c in data]
    return generate_bits(byte_data)

def bits_to_dna(data, conversion_table):
    """Convert binary bits to DNA sequence using a conversion table."""
    return "".join([conversion_table[bits] for bits in data])

def dna_to_bits(data, conversion_table):
    """Convert DNA sequence to binary bits using a conversion table."""
    return "".join([conversion_table[dna_base] for dna_base in data])
def dna_to_ascii(dna_str):
    # Mapping DNA bases to binary (2 bits per base)
    dna_mapping = {'A': '00', 'T': '01', 'C': '10', 'G': '11'}
    
    # Convert DNA string to binary string
    binary_str = ''.join(dna_mapping[base] for base in dna_str)
    
    # Ensure the binary string length is a multiple of 8
    if len(binary_str) % 8 != 0:
        binary_str = binary_str.ljust(len(binary_str) + (8 - len(binary_str) % 8), '0')
    
    # Convert binary string to ASCII string
    ascii_chars = [chr(int(binary_str[i:i+8], 2)) for i in range(0, len(binary_str), 8)]
    
    return ''.join(ascii_chars)

def ascii_to_dna(ascii_str):
    # Convert ASCII string to binary string
    binary_str = ''.join(f'{ord(char):08b}' for char in ascii_str)
    
    # Mapping binary to DNA bases (2 bits per base)
    binary_mapping = {'00': 'A', '01': 'T', '10': 'C', '11': 'G'}
    
    # Convert binary string to DNA bases
    dna_str = ''.join(binary_mapping[binary_str[i:i+2]] for i in range(0, len(binary_str), 2))
    
    return dna_str








def get_primer_pattern(delimiter, s):
    """Get patterns from a string based on a delimiter."""
    regex = f"{delimiter}(.*?){delimiter}"
    return re.findall(regex, s)

In [30]:
import string
from time import time


# number of rounds the algorithm is run, chosen randomly
ROUNDS_NUM = None

CHROMOSOME_LENGTH = None

# the key used in the decryption process
DECRYPTION_KEY = None


def set_globals():
    global ROUNDS_NUM
    global DECRYPTION_KEY
    # it is better to be odd random
    ROUNDS_NUM = 16
    DECRYPTION_KEY = ""



def encrypt_key(data, key):
 

    # repeat key ONLY if data is longer than key and encrypt
    if len(data) > len(key):
        factor = int(len(data) / len(key))
        key += key * factor

        return bitxor(data, key)

    return bitxor(data, key)



In [31]:

def rotate_crossover(population):
   
    global CHROMOSOME_LENGTH
    global DECRYPTION_KEY

    new_population = []

    DECRYPTION_KEY += ROTATE_CROSSOVER_PRIMER

    rotation_offset = random.randint(1, CHROMOSOME_LENGTH)

    DECRYPTION_KEY += ROTATION_OFFSET_PRIMER + str(rotation_offset) + ROTATION_OFFSET_PRIMER

    DECRYPTION_KEY += ROTATION_TYPE_PRIMER

    for chromosome in population:

        p = random.uniform(0, 1)

        if p > 0.5:
            DECRYPTION_KEY += "R|"
            right_first = chromosome[0: len(chromosome) - rotation_offset]
            right_second = chromosome[len(chromosome) - rotation_offset:]
            new_population.append(right_second + right_first)
        else:
            DECRYPTION_KEY += "L|"
            left_first = chromosome[0: rotation_offset]
            left_second = chromosome[rotation_offset:]
            new_population.append(left_second + left_first)

    DECRYPTION_KEY += ROTATION_TYPE_PRIMER

    DECRYPTION_KEY += ROTATE_CROSSOVER_PRIMER

    return new_population

In [32]:
def single_point_crossover(population):
   
    global DECRYPTION_KEY

    DECRYPTION_KEY += SINGLE_POINT_CROSSOVER_PRIMER

    new_population = []
    for i in range(0, len(population) - 1, 2):
        candidate1 = population[i]
        candidate2 = population[i + 1]

        # chromosomes have the same length
        # choose a random point
        length = len(candidate1)
        crossover_point = random.randint(0, length - 1)

        DECRYPTION_KEY += str(crossover_point) + "|"

        offspring1 = candidate2[0: crossover_point] + candidate1[crossover_point:]
        offspring2 = candidate1[0: crossover_point] + candidate2[crossover_point:]
        new_population.append(offspring1)
        new_population.append(offspring2)

    # append last chromosome if odd population size
    if len(population) % 2 == 1:
        new_population.append(population[len(population) - 1])

    DECRYPTION_KEY += SINGLE_POINT_CROSSOVER_PRIMER

    return new_population

In [33]:

def crossover(population):
    global DECRYPTION_KEY
    
    # choose crossover type according to p
    p = random.uniform(0, 1)

    if p < 0.33:
        DECRYPTION_KEY += CROSSOVER_TYPE_PRIMER + "rc" + CROSSOVER_TYPE_PRIMER
        return rotate_crossover(population)
    elif p >= 0.33 and p < 0.66:
        DECRYPTION_KEY += CROSSOVER_TYPE_PRIMER + "spc" + CROSSOVER_TYPE_PRIMER
        return single_point_crossover(population)
    else:
        DECRYPTION_KEY += CROSSOVER_TYPE_PRIMER + "b" + CROSSOVER_TYPE_PRIMER
        population = rotate_crossover(population)
        return single_point_crossover(population)

In [34]:
def complement(chromosome, POINT_A, POINT_B):

    new_chromosome = ""
    for i in range(len(chromosome)):
        if i >= POINT_A and i <= POINT_B:
            if chromosome[i] == '0':
                new_chromosome += '1'
            else:
                new_chromosome += '0'
        else:
            new_chromosome += chromosome[i]

    return new_chromosome

In [35]:
def alter_dna_bases(bases):
  
    alter_dna_table = {}

    for _ in range(2):
        # choose one randomly then remove it from list
        base1 = bases[random.randint(0, len(bases) - 1)]
        bases.remove(base1)

        # choose one randomly then remove it from list
        base2 = bases[random.randint(0, len(bases) - 1)]
        bases.remove(base2)

        # assign the first to the other
        alter_dna_table[base1] = base2
        alter_dna_table[base2] = base1

    return alter_dna_table

In [36]:
def mutation(population):

    global DECRYPTION_KEY

    bases = ['A', 'C', 'G', 'T']
    alter_dna_table = alter_dna_bases(bases)

    DECRYPTION_KEY += MUTATION_TABLE_PRIMER + str(alter_dna_table) + MUTATION_TABLE_PRIMER

    new_population = []
    for chromosome in population:
        DECRYPTION_KEY += CHROMOSOME_PRIMER

        # apply the complement
        b_chromosome = dna_to_bits(chromosome, dna_base_to_two_bits_table)
        DECRYPTION_KEY += COMPLIMENT_MUTATION_PRIMER
        POINT_A = random.randint(0, len(b_chromosome) - 1)
        POINT_B = random.randint(POINT_A, len(b_chromosome) - 1)
        DECRYPTION_KEY += "(%s,%s)" % (POINT_A,POINT_B)
        DECRYPTION_KEY += COMPLIMENT_MUTATION_PRIMER
        b_chromosome = complement(b_chromosome,POINT_A,POINT_B)

        # convert each 4 bits in chromosome to two dna bases using four_bits_to_two_dna_base_table
        four_bits_vector = group_bits(b_chromosome, 4)

        last_dna_base = None
        # if the last element is of length 2, don't convert it
        if len(four_bits_vector[len(four_bits_vector) - 1]) == 2:
            last_dna_base = two_bits_to_dna_base_table[four_bits_vector[len(four_bits_vector) - 1]]

            # convert only the 4 bits elements
            four_bits_vector = four_bits_vector[:-1]

        dna_seq = bits_to_dna(four_bits_vector, four_bits_to_two_dna_base_table)
        if last_dna_base is not None:
            dna_seq += last_dna_base

        # and then alter the dna bases between POINT_A and POINT_B
        DECRYPTION_KEY += ALTER_MUTATION_PRIMER
        POINT_A = random.randint(0, len(dna_seq) - 1)
        POINT_B = random.randint(POINT_A, len(dna_seq) - 1)
        DECRYPTION_KEY += "(%s,%s)" % (POINT_A,POINT_B)
        DECRYPTION_KEY += ALTER_MUTATION_PRIMER
        new_chromosome = ""
        for i in range(len(dna_seq)):
            if i >= POINT_A and i <= POINT_B:
                new_chromosome += alter_dna_table[dna_seq[i]]
            else:
                new_chromosome += dna_seq[i]

        new_population.append(new_chromosome)

        DECRYPTION_KEY += CHROMOSOME_PRIMER

    return new_population

In [37]:


def CHROMOSOME_PARTITIONING(dna_sequence):
    global CHROMOSOME_LENGTH
    global DECRYPTION_KEY

    # Choose the number of chromosomes as the first quartile of divisors
    divs = divisors(len(dna_sequence))
    midpoint = max(1, round(len(divs)/4))
    chromosome_no = random.randint(1, midpoint)
    CHROMOSOME_LENGTH = len(dna_sequence) // chromosome_no
    chromosomes = []

    DECRYPTION_KEY += PARTITIONING_PRIMER + str(CHROMOSOME_LENGTH) + PARTITIONING_PRIMER

    # Retrieve the population
    for i in range(0, len(dna_sequence), CHROMOSOME_LENGTH):
        chromosomes.append(dna_sequence[i:i + CHROMOSOME_LENGTH])

    return chromosomes


def REVERSE_CHROMOSOME_PARTITIONING(population):
    # convert the chromosome population back to DNA sequence
    return "".join(population)

In [38]:
def dna_get(text, key):
    global ROUNDS_NUM
    global DECRYPTION_KEY


    # binarize data and convert it to dna sequence
    b_data1 = binarized_data(text)
    dna_seq = bits_to_dna(b_data1, two_bits_to_dna_base_table)
    # print(dna_seq)


    b_data2 = dna_seq

    DECRYPTION_KEY += ROUNDS_NUM_PRIMER + str(ROUNDS_NUM) + ROUNDS_NUM_PRIMER

    # run the algorithm "ROUNDS_NUM" times
    while ROUNDS_NUM > 0:
        DECRYPTION_KEY += ROUNDS_CURRENT_PRIMER

        # encrypt data with key after reshaping it back to binary sequence and then convert it back to dna sequence
        b_data2 = bits_to_dna(
            group_bits(encrypt_key(dna_to_bits(REVERSE_CHROMOSOME_PARTITIONING(b_data2), dna_base_to_two_bits_table), key)),
            two_bits_to_dna_base_table)
        # print("Encrypted data:", b_data2)

        # create the chromosome population
        b_data2 = CHROMOSOME_PARTITIONING(b_data2)
        # print("Population data:", b_data2)

        # apply crossover on population
        DECRYPTION_KEY += GEN_CROSSOVER_PRIMER
        b_data2 = crossover(b_data2)
        DECRYPTION_KEY += GEN_CROSSOVER_PRIMER
        # print("Population data:", b_data2)

        # apply mutation on population

        DECRYPTION_KEY += MUTATION_PRIMER
        b_data2 = mutation(b_data2)
        DECRYPTION_KEY += MUTATION_PRIMER

        ROUNDS_NUM -= 1

        DECRYPTION_KEY += ROUNDS_CURRENT_PRIMER

    return REVERSE_CHROMOSOME_PARTITIONING(b_data2)


In [39]:
import os
os.listdir("/kaggle/input/")

['output1', 'lena-img', 'dnacrypto']

In [40]:
import shutil

src_path = r"/kaggle/input/dnacrypto/original.txt"

src_path1 = r"/kaggle/input/dnacrypto/key.txt"
src_path2 = r"/kaggle/input/dnacrypto/encrypted.txt"

dst_path = r"/kaggle/working/"

shutil.copy(src_path, dst_path)
shutil.copy(src_path1, dst_path)
shutil.copy(src_path2, dst_path)

print('Copied')

Copied


In [41]:
def main():
    global DECRYPTION_KEY

    original_file = open('/kaggle/working/original.txt', "w")
    text = "Grad-CAM (Gradient-weighted Class Activation Mapping) provides visual explanations for decisions made by CNNs by highlighting the regions in an input image that are most influential for a given prediction. It achieves this by computing the gradient of the target class score with respect to the feature maps of the last convolutional layer, essentially capturing the importance of each spatial location. These gradients are then used to weight the feature maps, producing a localization map that highlights the significant regions. The resulting heatmap, superimposed on the original image, reveals the areas that the model focuses on, thus aiding in model interpretability and debugging by pinpointing the features that drive the CNN's decisions."
    
    print(len(text))
    #text = ''.join(random.SystemRandom().choice(string.ascii_letters + string.digits + string.punctuation + string.whitespace) for _ in range(5000))

    print("Text:", text)

    original_file.write(text)

   
    key = str2bin(''.join(random.SystemRandom().choice(string.ascii_letters + string.digits) for _ in range(16)))

    print("Key:", len(key), key)

    # set initial values of global variables
    set_globals()

    # append the key first
    DECRYPTION_KEY += KEY_PRIMER + key + KEY_PRIMER
    print(DECRYPTION_KEY)
    # generate the encoding tables
    generate_pre_processing_tables()
    generate_mutation_tables()
    
    # get the ciphertext
    start = time()
    encrypted_text = dna_get(text, key)
    print(encrypted_text)
    #encrypted_text=dna_to_ascii(encrypted_text)
    
    print("Final  sequence:", encrypted_text)

   
    end = time()
    
    print("\nTotal execution time:", end - start)
 
    key_file = open('/kaggle/working/key.txt', "w")
    encrypted_file = open('/kaggle/working/encrypted.txt', "w")
 
    # save the encryption to a file to be used in the decryption process
    encrypted_file.write(encrypted_text)
    print(DECRYPTION_KEY)
    # save key to a file to be read in the decryption process
    key_file.write(DECRYPTION_KEY)

    encrypted_file.close()
    original_file.close()
    key_file.close()


   
if __name__ == '__main__':
    main()
   

707
Text: Grad-CAM (Gradient-weighted Class Activation Mapping) promade by CNNs by highlighting the regions in an input image that are most influential for a given prediction. It achieves this by computing the gradient of the target class score with respect to the feature maps of the last convolutional layer, essentially capturing the importance of each spatial location. These gradients are then used to weight the feature maps, producing a localization map that highlights the significant regions. The resulting heatmap, superimposed on the original image, reveals the areas that the model focuses on, thus aiding in model interpretability and debugging by pinpointing the features that drive the CNN's decisions.
Key: 128 01101011011011000111011101000010011010100110100001001011011011100110001001011010011110000111010001101110010011100101010100110010
k_01101011011011000111011101000010011010100110100001001011011011100110001001011010011110000111010001101110010011100101010100110010k_
AGATATCCTAA

In [42]:

from time import time
import ast

import string
import re

def encrypt_key(data, key):

    if len(data) > len(key):
        factor = int(len(data) / len(key))
        key += key * factor

        return bitxor(data, key)

    return bitxor(data, key)

def CHROMOSOME_PARTITIONING(dna_sequence, CHROMOSOME_PARTITIONING_info):
  

    CHROMOSOME_LENGTH = int(CHROMOSOME_PARTITIONING_info[0])
    chromosomes = []

    # retrieve the population
    for i in range(0, len(dna_sequence), CHROMOSOME_LENGTH):
        chromosomes.append(dna_sequence[i:i + CHROMOSOME_LENGTH])

    return chromosomes


def REVERSE_CHROMOSOME_PARTITIONING(population):
    # convert the chromosome population back to DNA sequence
    return "".join(population)


def rotate_crossover(population, rotate_info):


    new_population = []

    # get the rotation value
    rotation_offset = int(get_primer_pattern(ROTATION_OFFSET_PRIMER, rotate_info)[0])
    
    rotations = get_primer_pattern(ROTATION_TYPE_PRIMER, rotate_info)[0].split("|")[:-1]

    for i in range(len(population)):
        chromosome = population[i]

        direction = rotations[i]

        if direction == "L":
            right_first = chromosome[0: len(chromosome) - rotation_offset]
            right_second = chromosome[len(chromosome) - rotation_offset:]
            new_population.append(right_second + right_first)
        elif direction == "R":
            left_first = chromosome[0: rotation_offset]
            left_second = chromosome[rotation_offset:]
            new_population.append(left_second + left_first)

    return new_population

def single_point_crossover(population, single_point_info):
    crossover_points = [int(p) for p in single_point_info.split("|") if p != '']

    new_population = []
    for i in range(0, len(population) - 1, 2):
        candidate1 = population[i]
        candidate2 = population[i + 1]

        # get the crossover_point
        crossover_point = crossover_points[int(i / 2)]

        offspring1 = candidate2[0: crossover_point] + candidate1[crossover_point:]
        offspring2 = candidate1[0: crossover_point] + candidate2[crossover_point:]
        new_population.append(offspring1)
        new_population.append(offspring2)

    # append last chromosome if odd population size
    if len(population) % 2 == 1:
        new_population.append(population[len(population) - 1])

    return new_population


def crossover(population, crossover_info):
    # get the crossover type
    crossover_type = get_primer_pattern(CROSSOVER_TYPE_PRIMER, crossover_info)[0]

    if crossover_type == "rc":
        rotate_info = get_primer_pattern(ROTATE_CROSSOVER_PRIMER, crossover_info)[0]
        return rotate_crossover(population, rotate_info)
    elif crossover_type == "spc":
        single_point_info = get_primer_pattern(SINGLE_POINT_CROSSOVER_PRIMER, crossover_info)[0]
        return single_point_crossover(population, single_point_info)
    elif crossover_type == "b":
        rotate_info = get_primer_pattern(ROTATE_CROSSOVER_PRIMER, crossover_info)[0]
        single_point_info = get_primer_pattern(SINGLE_POINT_CROSSOVER_PRIMER, crossover_info)[0]
        population = single_point_crossover(population, single_point_info)
        return rotate_crossover(population, rotate_info)



def complement(chromosome, POINT_A, POINT_B):
   
    new_chromosome = ""

    for i in range(len(chromosome)):
        if i >= POINT_A and i <= POINT_B:
            if chromosome[i] == '0':
                new_chromosome += '1'
            else:
                new_chromosome += '0'
        else:
            new_chromosome += chromosome[i]

    return new_chromosome





def mutation(population, mutation_info):
  
    # extract the alteration table
    alter_dna_table = ast.literal_eval(get_primer_pattern(MUTATION_TABLE_PRIMER, mutation_info[0])[0])

    chromosomes_info = get_primer_pattern(CHROMOSOME_PRIMER, mutation_info[0])
    new_population = []
    for i in range(len(population)):
        chromosome = population[i]
        chromosome_info = chromosomes_info[i]

        # alter back the dna bases between POINT_A and POINT_B
        alter_info = get_primer_pattern(ALTER_MUTATION_PRIMER, chromosome_info)[0]
        POINT_A, POINT_B = ast.literal_eval(alter_info)
        new_chromosome = ""
        for i in range(len(chromosome)):
            if i >= POINT_A and i <= POINT_B:
                new_chromosome += alter_dna_table[chromosome[i]]
            else:
                new_chromosome += chromosome[i]

        two_bases_vector = group_bases(new_chromosome)

        # last base was not converted using four_bits_to_two_dna_base_table
        # convert it to bits using dna_base_to_two_bits_table
        last_two_bits = None
        if len(new_chromosome) % 2 == 1:
            last_two_bits = dna_base_to_two_bits_table[new_chromosome[-1]]

            two_bases_vector = two_bases_vector[:-1]

        bits_seq = dna_to_bits(two_bases_vector, two_dna_base_to_four_bits_table)

        if last_two_bits is not None:
            bits_seq += last_two_bits

        complement_info = get_primer_pattern(COMPLIMENT_MUTATION_PRIMER, chromosome_info)[0]
        POINT_A, POINT_B = ast.literal_eval(complement_info)
        b_chromosome = complement(bits_seq, POINT_A, POINT_B)
        b_chromosome = group_bits(b_chromosome)
        new_chromosome = bits_to_dna(b_chromosome,two_bits_to_dna_base_table)

        new_population.append(new_chromosome)

    return new_population



def dna_gdt(text, key):
    print("\nDNA-GDT is running...\n")
    ROUNDS_NUM_pattern = get_primer_pattern(ROUNDS_NUM_PRIMER, key)
    if not ROUNDS_NUM_pattern:
        print("Error: No pattern found for the number of rounds.")
        return None
    
    ROUNDS_NUM = int(ROUNDS_NUM_pattern[0])
    rounds = get_primer_pattern(ROUNDS_CURRENT_PRIMER, key)
   

    b_data = text
    print("Initial DNA sequence:", b_data)

    # run the algorithm "ROUNDS_NUM" times
    while ROUNDS_NUM > 0:
        round_info = rounds[ROUNDS_NUM - 1]
        
        # create the chromosome population
        b_data = CHROMOSOME_PARTITIONING(b_data, get_primer_pattern(PARTITIONING_PRIMER, round_info))

        # apply mutation on population
        b_data = mutation(b_data, get_primer_pattern(MUTATION_PRIMER, round_info))

        # apply crossover on population
        b_data = crossover(b_data, round_info)

        # decrypt = encrypt(encrypt(data, key), key) and encrypt => xor operation, because (a xor b) xor b = a
        encryption_key = get_primer_pattern(KEY_PRIMER, key)[0]
        b_data = bits_to_dna(
            group_bits(
                encrypt_key(dna_to_bits(REVERSE_CHROMOSOME_PARTITIONING(b_data), dna_base_to_two_bits_table), encryption_key)),
            two_bits_to_dna_base_table)
        # print("Decrypted data:", b_data)

        ROUNDS_NUM -= 1
    return bin2str(dna_to_bits(b_data, dna_base_to_two_bits_table))









def main():
    original_file_path='/kaggle/working/original.txt'
    encrypted_file='/kaggle/working/encrypted.txt'
    key_file='/kaggle/working/key.txt'
    with open('/kaggle/working/original.txt', "r",encoding='utf-8', errors='replace') as original_file:
        original_text = original_file.read()
        
    
    with open('/kaggle/working/encrypted.txt', "r",encoding='utf-8', errors='replace') as encrypted_file:
        encrypted_text = encrypted_file.read()

    with open('/kaggle/working/key.txt', "r") as key_file:
        key = key_file.read()
    print("Encrypted text:", encrypted_text)
 

    # generate the encoding tables
    generate_pre_processing_tables()
    generate_mutation_tables()
    
    # get the decrypted text
    start = time()
    #decrypted_text=ascii_to_dna(encrypted_text)
    #print(decrypted_text)
    decrypted_text = dna_gdt(encrypted_text, key)
    end = time()
    print(decrypted_text)
    print("\nTotal execution time:", end - start)

    if original_text != decrypted_text:
        print("\nDecryption failed.")
    else:
        print("\nDecryption succeeded.")

    original_file.close()
    encrypted_file.close()
    key_file.close()


if __name__ == '__main__':
    main()


Encrypted text: AGATATCCTAATCCCGCCTTCAAAGTAATTATGCCTCACCTGGGTTTCCCAGGACCGAGGATGAATCAGAGTCGGACGGCCGCAAGTGGATAACTTTAAGCACTGGGCAGGACAAATACGCAGGGTTAAGACGCGAACCGACACACTCCTGGACGGCCGCACATGCCAAACGGACGGTGCTGGCGGCGTCCTGCGAAGACTAACAAATCAGGCCCCTGGACAGGGCATTCAGGTGCGTGCCACCTTTAGGAAGCATCCGGCTAATAAGTAAACAGTTACTGGCAAGCGTTTGCTTTAAGCAGGACAAATACGCATGTTTTCTATTCGGAGTTGGGTCGAAGTTAGTCTGTAGATGGTTTGGACGGCAGAGGCCGGCTTACAAACCATTAACGCCACTGTAGCGACATACGGTTATTTACAGTATCCGACATGCCCCGGCAATTTAACGTGCGCCCTGTGCCTATGACTTCGCGCCACCACGAGCTCATTAAGTTAGACAACTCACAGTCATTGAGCTCTTGTTCTCGAAAGCGTGTAGACACAGCCGATTCGTCAGTGGTCGCAATTTGCGTATGAGATATATTGTACCGCTAATGTACTGCACTCCAAAACCACGGGGGGCGGGATCGATTCCAGCGGGGCTAGCGTTGAGGCGCTCCCATTACCCCTCCAAAACCACGGCCCCGCCATTGATTATGACGCGCCCATAGACACGGTACATGCCAACAAAAAGCGCCCCGTCGGGATACCGCAGCGAGTCGGGCTCGCCGCATCTTTACAATCTCGAGGCCAAAAAGCCGTGTTTAAACTGTTACGCAGGATCCGCGAACCAGGCGTAATACGCTGTGAGAGGTTTTCCAGCAGCTCCGCCTCTTCACTCAGACCATTATCTGTGCCTGTCATTACCCCTACTCGCTAAGTACTCCAATAGACGGACAACGTACTCTACTCTCCGGCAGTCAAGGACTACAACTACTCTACACG